In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as c
import astropy.units as u

In [3]:
def Planck(lam,T):
    
    h = 6.626e-34
    c = 299792458
    k_B = 1.38064852e-23
    A = (2*h*c**2)/(lam**5)
    B = np.exp(h*c/(lam*k_B*T))
    planck = A*1/(B-1)
    
    return planck

In [4]:
def Equilibrium_Temp(AB, axis, tstar):
    
    Teq = (1-AB)**(0.25)*(2*axis)**(-0.5)*(tstar) 
    
    return Teq

In [5]:
def Insolation(axis, tstar):
    
    S = (tstar/5772)**4*(215/axis)**2
    
    return S

In [6]:
def Mp_Chen(Re): # Chen & Kipping 2017
    
    Mp = 1.436*Re**1.7 # Rp in earth radii, this is for planets with R>1.23 R_earth
    
    return Mp

In [7]:
def ESM(Teq,Teff,Rp_Rs,m_K): #Kempton et al 2018
    
    ESM = 4.29e6*Planck(7.5e-6,1.1*Teq)/Planck(7.5e-6,Teff)*Rp_Rs**2.0*10**(-m_K/5)
    
    return ESM

In [8]:
def TSM(Re, Teq, mj, Rs, Me, scale_factor):

    TSM = scale_factor * (Re**3*Teq)/(Me*Rs**2)*10**(-mj/5)
    
    return TSM

In [9]:
def Escape_Velocity(Me,Re,Ms,a_Rs,Rs,Teq):
    
    # Mp in kg and Rp in m
    
    Rp = Re*c.R_earth
    Mp = Me*c.M_earth
    vesc = np.sqrt(2*c.G*Mp/Rp)
    
    return vesc

In [10]:
def Thermal_Velocity(Ms,a_Rs,Rs,Teq):
    
    # This is the thermal velocity for a proton
        
    Ms = Ms*c.M_sun
    r = a_Rs*Rs*c.R_sun
    v_therm = np.sqrt(3*(c.k_B*1.1*Teq*u.K/c.m_p))
    
    return v_therm

In [11]:
def tidal_circ(Period,Me,Ms,Rp_Rs,a_Rs,Q):
    
    # Period in whatever units, everything else unitless
    Mp = Me*c.M_earth
    Ms = Ms*c.M_sun
    tau = 2*Period*Q/(63*np.pi)*(Mp/Ms)*(a_Rs/Rp_Rs)**5
    
    return tau/365/1e9 # years

In [12]:
def luminosity(Teff,Rs):
    
    L = (Teff/5778)**4*(Rs)**2
    
    return L

In [13]:
def rho_star(a_r,period):
    
    rho = (3*np.pi)/(c.G*((period*u.s)*24.0*60*60)**2)*(a_r)**3.0
    
    return rho # kg/m^3

In [14]:
def semi_major_axis(Ms, period):
    Ms = Ms*c.M_sun
    a = ((c.G*Ms*((period*u.s)*24.0*60*60)**2)/(4*np.pi**2))**(1/3)
    a = a/1.496e11*u.au/u.m
    return a

## TOI 122

In [14]:
samples = np.load('TOI122_LCO_samples.dat.npy')
A1,A2,A3,A4,A5,A7,A8,A9,B1,B2,B3,B4,B5,B7,B8,B9,t1,t2,t3,t4,t8, Rp_Rs, a_Rs, inc = samples

M_s = np.random.normal(0.312,0.007,len(a_Rs))
# Msun, from Mann+ 2019, calculated in Constrain_your_Mdwarf.ipynb

R_s = np.random.normal(0.334,0.01,len(a_Rs))
# Mann+ 2015

Teff = np.random.normal(3403, 100, len(a_Rs))
# K, from Mann+ 2015

period = np.random.normal(5.078030, 0.000015, len(a_Rs))
# days, from modeling the light curves and fitting a line

a_in_AU = (M_s*(period/365.26)**2)**(1/3) #AU

aRs_from_P = a_in_AU/R_s*(1/6.957e8)*(1.496e11)

m_K = 10.771
# from ExoFOP-TESS

In [32]:
np.percentile(aRs_from_P,[16.,50.,84.])

array([23.9528886, 25.2400895, 26.6559659])

In [15]:
insolation = Insolation(axis = aRs_from_P, tstar=Teff)
np.percentile(insolation, [16.,50.,84.])

array([7.65793621, 8.75385979, 9.97457243])

In [16]:
rv_amp = 28.4329*(8.8/317.8)*np.sin(88.4*np.pi/180)*(0.312)**(-2/3)*(5.078030/365.26)**(-1/3)
print(rv_amp)

7.115080211943929


In [18]:
g = np.log10((c.G*M_s*c.M_sun/(R_s*c.R_sun)**2.0).value*100)
np.percentile(g,[16.,50.,84.])

array([4.85739253, 4.88467746, 4.91264467])

In [116]:
density_model_samples = rho_star(a_Rs,5.078030)

print('Stellar Density (kg/m3)')
np.percentile(density_model_samples, [16., 50., 84.])

Stellar Density (kg/m3)


array([ 8633.33579117, 12766.59395378, 22253.04277632])

In [117]:
Mstar = Ms*c.M_sun #kg
Rstar = (((3*Mstar)/(4*np.pi*density_model_samples))**(1/3))/c.R_sun
print('Stellar Radius (Rsun)')
np.percentile(Rstar, [16., 50., 84.])

Stellar Radius (Rsun)


array([0.27007419, 0.32513439, 0.37095714])

In [123]:
Mstar = Ms*c.M_sun #kg
Rstar = (((3*Mstar)/(4*np.pi*density_model_samples))**(1/3))
stellar_radius = Rs*c.R_sun/1.496e11*u.au/u.m
np.percentile(stellar_radius, [16., 50., 84.])

array([0.00147464, 0.00155323, 0.00163187])

In [126]:
print('Planet Radius (Re)')
Re = Rp_Rs*Rs*109.168
np.percentile(Re, [16., 50., 84.])

Planet Radius (Re)


array([2.5531984 , 2.73095147, 2.91211846])

In [128]:
print('Luminosity (L_sun)')
np.percentile(luminosity(Teff,Rs), [16., 50., 84.])

Luminosity (L_sun)


array([0.01123444, 0.01274029, 0.01439211])

In [21]:
print('Teq (K)')
Equilibrium_Temp(AB=0.75,axis=26.1,tstar=3403)

Teq (K)


333.0517345014227

In [22]:
therm = Thermal_Velocity(Ms=M_s,a_Rs=a_Rs,Rs=R_s,Teq=Equilibrium_Temp(AB=0.5,axis=a_Rs,tstar=Teff))
np.percentile(therm,[16., 50., 84.])

<Quantity [3146.6354088 , 3295.08332859, 3413.97874776] J(1/2) / kg(1/2)>

In [140]:
esc = Escape_Velocity(Me=np.random.normal(8.8,3.0,len(a_Rs)),Re=2.72,Ms=M_s,a_Rs=a_Rs,Rs=R_s,
                      Teq=Equilibrium_Temp(AB=0.5,axis=a_Rs,tstar=Teff))
np.nanpercentile(esc,[16., 50., 84.])

array([16372.69459518, 20117.49832541, 23272.92201306])

In [141]:
jeans = np.random.normal(20117,3300,10000)/np.random.normal(3250,150,10000)
np.percentile(jeans,[16., 50., 84.])

array([5.12672553, 6.19090069, 7.24975448])

In [23]:
ESM(Teq=Equilibrium_Temp(AB=0.5,axis=26.1,tstar=3403),
    Teff=3403,Rp_Rs=0.0748,m_K=10.771)

1.5788693283717024

In [142]:
TSM(Re=2.72, Teq=425, mj=11.531 , Rs=0.334, Me=8.8, scale_factor=1.26)

54.23636583798516

In [143]:
print('Tidal Circularization Timescale (Gyr):')
tide = tidal_circ(Period=5.078030,Me=8.8,Ms=M_s,
           Rp_Rs=0.075,a_Rs=a_Rs,Q=1e4)
np.percentile(tide, [16.,50.,84.])

Tidal Circularization Timescale (Gyr):


array([0.30540579, 0.58771958, 1.4842053 ])

In [144]:
print('Surface Gravity (m/s2):')
9.8*8.8/(2.72**2)

Surface Gravity (m/s2):


11.656574394463666

In [32]:
# Mann Density
mass = np.random.normal(0.312,0.05*0.312,10000)*c.M_sun
radius = np.random.normal(0.334, 0.05*0.334, 10000)*c.R_sun
density = mass/(4/3*np.pi*radius**3)
np.percentile(density, [16.,50.,84.])

array([10070.66980721, 11727.01673949, 13789.81483459])

## TOI 237

In [24]:
samples = np.load('TOI237_LCO_samples.dat.npy')
A1,A2,A3,A4,A5,A6,B1,B2,B3,B4,B5,B6,t1,t2,t3,t5,t6 ,Rp_Rs, a_Rs, inc = samples

Ms = np.random.normal(0.179, 0.004, len(a_Rs))
# Msun, from Mann+ 2019, calculated in Constrain_your_Mdwarf.ipynb

Rs = np.random.normal(0.211,0.006,len(a_Rs))

Teff = np.random.normal(3212, 100, len(a_Rs))
# K, from Magellan & SALT spectra

period = np.random.normal(5.436098, 0.000039, len(a_Rs)) # days
# days, from modeling the light curves and fitting a line

a_in_AU = (Ms*(period/365.26)**2)**(1/3) #AU

aRs_from_P = a_in_AU/Rs*(1/6.957e8)*(1.496e11)

m_K = 10.896
# from ExoFOP-TESS

In [15]:
Ms = np.random.normal(0.361, 0.008, 100000)
# Msun, from Mann+ 2019, calculated in Constrain_your_Mdwarf.ipynb

Rs = np.random.normal(0.377,0.011,100000)

Teff = np.random.normal(3566, 100, 100000)
# K, from Magellan & SALT spectra

period = np.random.normal(18.7913430, 0.0004, 100000) # days
# days, from modeling the light curves and fitting a line

a_in_AU = (Ms*(period/365.26)**2)**(1/3) #AU

aRs_from_P = a_in_AU/Rs*(1/6.957e8)*(1.496e11)

In [16]:
np.percentile(aRs_from_P,[16.,50.,84.])

array([54.52839419, 56.16708328, 57.89757908])

In [25]:
insolation = Insolation(axis = aRs_from_P, tstar=Teff)
np.percentile(insolation, [16.,50.,84.])

array([3.19241667, 3.66738344, 4.19896892])

In [38]:
rv_amp = 28.4329*(3/317.8)*np.sin(89.5*np.pi/180)*(0.179)**(-2/3)*(5.436098/365.26)**(-1/3)
print(rv_amp)

3.435380986448534


In [36]:
np.sin(np.pi/2)

1.0

In [26]:
g = np.log10((c.G*Ms*c.M_sun/(Rs*c.R_sun)**2.0).value*100)
np.percentile(g,[16.,50.,84.])

array([5.01618788, 5.0423069 , 5.06899413])

In [147]:
density_model_samples = rho_star(a_Rs,period)
print('Stellar Density (kg/m3)')
np.percentile(density_model_samples, [16., 50., 84.])

Stellar Density (kg/m3)


array([16606.97458435, 25547.01983455, 30001.24880897])

In [153]:
print('Planet Radius (Re)')
Re = Rp_Rs*Rs*109.168
np.percentile(Re, [16., 50., 84.])

Planet Radius (Re)


array([1.32137969, 1.4385532 , 1.56008086])

In [93]:
print('Luminosity (L_sun)')
np.percentile(luminosity(Teff,Rs), [16., 50., 84.])

Luminosity (L_sun)


array([0.00334203, 0.00397565, 0.00468605])

In [29]:
print('Teq (K)')
Equilibrium_Temp(AB=0,axis=34.2,tstar=3212)

Teq (K)


388.3716192255596

In [155]:
therm = Thermal_Velocity(Ms=Ms,a_Rs=a_Rs,Rs=Rs,
                 Teq=Equilibrium_Temp(AB=0.5,axis=a_Rs,tstar=Teff))

In [156]:
esc = Escape_Velocity(Me=np.random.normal(3.0,2.0,len(a_Rs)),Re=np.random.normal(1.44,0.12,len(a_Rs)),Ms=Ms,a_Rs=a_Rs,Rs=Rs,
                Teq=Equilibrium_Temp(AB=0.5,axis=a_Rs,tstar=Teff))

In [158]:
np.nanpercentile((esc/therm),[16.,50.,84.])

array([3.71732847, 5.5449992 , 7.06979875])

In [99]:
ESM(Teq=Equilibrium_Temp(AB=0.5,axis=34.2,tstar=3160),
    Teff=3160,Rp_Rs=0.062,m_K=m_K)

0.40211435527845996

In [100]:
TSM(Re=1.44, Teq=382, mj=11.74 , Rs=0.211, Me=2.6, scale_factor=0.19)

8.401680876559114

In [49]:
print('Tidal Circularization Timescale (Gyr):')
tidal_circ(Period=5.436098,Me=2.6,Ms=0.179,
           Rp_Rs=0.062,a_Rs=34.2,Q=500)

Tidal Circularization Timescale (Gyr):


<Quantity 0.16765731>

In [159]:
print('Surface Gravity (m/s2):')
9.8*3.0/(1.44**2)

Surface Gravity (m/s2):


14.178240740740742

In [51]:
# Mann Density
mass = np.random.normal(0.179,0.05*0.179,10000)*c.M_sun
radius = np.random.normal(0.212, 0.05*0.212, 10000)*c.R_sun
density = mass/(4/3*np.pi*radius**3)
np.percentile(density, [16.,50.,84.])

array([22677.36659202, 26414.01076269, 31079.35978546])

## Rotational Period from vsini

In [ ]:
# R = 0.322*c.R_sun/1000*u.km/u.m
# v = 6*u.km/u.s

R = 0.214*c.R_sun/1000*u.km/u.m
v = 7*u.km/u.s
period = (2*np.pi*R/v) * (u.min/(60*u.s)) * (u.hour/(60*u.min)) * (u.day/(24*u.hour))
print(period)